In [22]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

# Python functions

In [21]:
# The function name, type hints, and docstring are all part of the tool
# schema that's passed to the model. Defining good, descriptive schemas
# is an extension of prompt engineering and is an important part of
# getting models to perform well.
def add(a: int, b: int) -> int:
    """Add two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a + b


def multiply(a: int, b: int) -> int:
    """Multiply two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a * b

# Tool calls

In [23]:
llm = ChatOpenAI(model="qwen-plus")

In [24]:
tools = [add, multiply]
llm_with_tools = llm.bind_tools(tools)

query = "What is 3 * 12?"

message = llm_with_tools.invoke(query)
message.tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 12},
  'id': 'call_22e65767eb424df0834743',
  'type': 'tool_call'}]

# Parsing

In [25]:
from langchain_core.output_parsers import PydanticToolsParser
from pydantic import BaseModel, Field


class add(BaseModel):
    """Add two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class multiply(BaseModel):
    """Multiply two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


chain = llm_with_tools | PydanticToolsParser(tools=[add, multiply])
res = chain.invoke(query)

In [27]:
llm_with_tools.invoke(query)

AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 255, 'total_tokens': 280, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 128}}, 'model_provider': 'openai', 'model_name': 'qwen-plus', 'system_fingerprint': None, 'id': 'chatcmpl-187f3712-a744-4a2e-a21c-10c08a2d3a3e', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--f2710eca-a8d4-441f-aed4-48a02466e293-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_bc76bc61f8cb4d8999c488', 'type': 'tool_call'}], usage_metadata={'input_tokens': 255, 'output_tokens': 25, 'total_tokens': 280, 'input_token_details': {'cache_read': 128}, 'output_token_details': {}})